# Cosimulation Documentation Notebook
This notebook serves as documentation and testing for the cosimulation and `ExternalState` blocks in Sedaro. This notebook starts a simulation of the Wildfire scenario and interacts with it through the cosimulation interface.

## Setup


#### Important: Read Before Running

These notebooks may make changes to agent and scenario branches in your account. Ensure any changes to the target branches are saved prior to running any code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

These notebooks also require that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as these notebooks with the following format:

```json
{
  "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.


In [73]:
import json

with open('../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('../config.json', 'r') as file:
    config = json.load(file)

# Obtain these IDs from the branch list within each repository and add to config.json
SCENARIO_BRANCH_ID = config['BASICSAT']['SCENARIO_BRANCH_ID']                # ID of the scenario branch
BASICSAT1_ID = "PQQtlQggyzHgX2y9zyT5Kw"                                      # Wildfire ID (should not need changing)
BASICSAT2_ID = "PQQtlyCFycdBcPR7TDJvDq"
HOST = config['HOST']                                                        # Sedaro instance URL
WEB_HOST = config['WEB_HOST']                                                # Sedaro web URL


# Cosimulation in Sedaro
In Sedaro, *cosimulation* refers to the ablity of a running simulation to interact with hardware, software, and other simulations outside Sedaro. This is achieved through the creation of cosimulation [blocks](https://docs.sedaro.com/KeyConcepts/blocks), which use the query language to specify a set of consumed and produced values for a given cosimulation interface.

## Cosimulation modes
Each cosimulation interface is either *per-round*, meaning it consumes simulation data and produces a new output at every timestep, or *spontaneous*, meaning that it can produce and consume state at any time. If the simulator state depends on data provided by a per-round cosimulation interface, the simulation will block until the cosimulator returns. 

Spontaneous cosimulation is recommended for real-time simulations. Produced spontaneous state is timestamped, and will affect the simulation at the round at or immediately following the timestamp. Spontaneous state production is also unique in that it is not required to ever provide a value and can remain optionally unused during a simulation. In this case, the simulation will continue to consume the initial value for the state until a new value is provided.

Each cosimulation interface is independent meaning some may be Per Round while others are Spontaneous. This also means that individual cosimulators can consume/produce state simultaneously over different interfaces.

## Connecting to the Sedaro API
To connect to the Sedaro API, you need to have generated an API key in the web UI for your instance. API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

Once you have the API key, you can create a SedaroApiClient object which will connect to the API and handle future communication.

In [74]:
from sedaro import SedaroApiClient

# We recommend loading your API key from an environment variable or configuration file.
sedaro = SedaroApiClient(host=HOST, api_key=API_KEY)

## Selecting and interacting with a scenario branch
Cosimulation blocks are defined at the scenario level, which allows them to be used without modifying agent templates that may be used across multiple scenarios. To add cosimulation blocks to your scenario, you'll need to know the id of the branch you want to modify. For this example, we'll be starting with the BasicSat demo scenario. You may wish to set it to run in real time.

In [75]:
# Replace SCENARIO_BRANCH_ID with an ID copied from the web UI
scenario = sedaro.scenario(SCENARIO_BRANCH_ID)

In [76]:
cosim_blocks = dict()

## Creating a cosimulation block
New blocks can be added to the scenario via the `scenario` object created above. The `scenario` object contains a field for each concrete block type available on scenarios, and those have a `create` method that fills in the data for a specific block and returns a handle to it. Each cosimulation block specifies a set of agents to run on, what engine to use within those agents, and a set of consumed inputs and produced outputs. The agents are specified as a list of IDs, the engine by its name, and the inputs and outputs by query strings.

# SedaroQL queries


## Accessing blocks and state variables

### Specifying a block by its ID
To get a specific block, use the `block!(id)` syntax, with the block's ID as a quoted string. By itself, this will result in the block's ID, but can also be used with other operators that expect a block.

### Accessing state from a block
To access state from a block, use the `.` operator. When the left-hand side is a block and the right-hand side is the name of a field, it will get the value of that field.

In [ ]:
cosim_blocks["active_subroutine"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        block!("PLCZKQWpjfNLcRbLg65yw6").activeSubroutine,
    )''',
    produced='''()''',
    engine='cdh',
    agents=[BASICSAT1_ID]
)

### Accessing state from the agent root
Certain fields such as position and velocity are part of the agent's root block. To access the root block, use the `root!` keyword.

In [ ]:
cosim_blocks["root_position_velocity"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        root!.position,
        root!.velocity
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Specifying a block via a relation
When the `.` syntax is used with the name of a relation rather than a field, it results in the block that is the current value of that relation. For a many-sided relation, the result will be a list of blocks.

In [ ]:
cosim_blocks["pointing_mode_name"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        root!.activePointingMode.name,
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Specifying all blocks of a type
To specify all blocks of a type, use the block type's name. This will result in a list containing all blocks of that type.

### Accessing state from a list of blocks
When the left side of a `.` access specifies a list of blocks, the result will be the list of values derived from loading that field from each of the specified blocks. For instance, the following query will get a list containing the speed of each reaction wheel on the BasicSat 1 agent.

In [ ]:
cosim_blocks["reaction_wheel_speeds"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        ReactionWheel.speed,
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

List of lists

### Converting units and reference frames
To convert scalar quantitied between units or vectors between reference frames, use the `as` keyword. The left side of the `as` is the value to convert, and the right side is the unit or reference frame to convert to.

In [ ]:
cosim_blocks["position_in_ecef"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        root!.position as Position.ecef,
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Reading values from a dictionary

In [34]:
cosim_blocks["orbital_eccentricity"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        root!.orbitalElements["e"],
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

## Conditional queries

### Conditional queries with if/else
The if/else syntax allwos you to pick between two result queries depending on the current simulation state. The condition must be a query that results in a boolean.

In [ ]:
cosim_blocks["iss_target_range"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        if block!("PQQtQRfgr45p9Y3bv6SqH2").lineOfSight {
            block!("PQQtQRfgr45p9Y3bv6SqH2").range
        } else {
            null
        },
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Conditional queries with dynamic match
The `match` syntax allows you to check a dynamic value against a list of match pairs. Each *match pair* consists of a *pattern* and a *result query*. Patterns currently consist of quoted string constants, which match identical strings, and the `_` pattern, which will match with any value. The result of the match will be the result query of the first match pair with a pattern that matches the argument.

In [ ]:
cosim_blocks["pointing_mode"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        match root!.activePointingMode.name {
            "ISS Pointing Mode" => "ISS",
            "Other BasicSat Pointing Mode" => "BasicSat",
            _ => "Unknown"
        },
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Matching against a field statically
The `static match` syntax works similarly to the dynamic `match` syntax, but requires that the argument be evaluated statically. This allows better optimization in some cases.

In [ ]:
cosim_blocks["celestial_vector"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        static match CelestialVector.celestialPointingDirection {
            "SUN" => "The Sun",
            "MOON" => "The Moon",
            _ => "A Planet"
        },
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Conditioning on the type of a block
In some cases, the fields you want to access may only be available on certain subtypes of a block type. In this case, a dynamic match on the type of the block won't work. Instead, use the `type match` syntax, which uses block type names instead of patterns. Additionally, the results in a type match use spread elements rather than query expressions.

In [ ]:
cosim_blocks["actuator_ratings"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        type match Actuator {
            ReactionWheel => (
                name,
                "ratedTorque",
                ratedTorque
            ),
            Magnetorquer => (
                name,
                "ratedMagneticMoment",
                ratedMagneticMoment
            )
        },
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

## Spread Syntax
Certain forms of queries use *spread elements*, which allow multiple operations to be applied to a single *base expression*. Not all operations available as expressions are available as spread elements.

Informally, you can think of a spread element as being like an expression missing its leftmost operand, which will be supplied by the surrounding *spread context*.

### Spread Expressions
The most common form of expression that uses spread elements is the *spread expression* `base.(spread1, spread2)`. In a spread expression, the base expression is the left-hand side of the `.`, and the spread elements appear in parentheses, separated by commas. The result of a spread expression is a tuple in which each spread element is separately applied to the base expression.

In [ ]:
cosim_blocks["reaction_wheel_spread"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        ReactionWheel.(commandedTorqueMagnitude, achievedTorqueMagnitude),
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Spread Accesses
In the query `base.(field1, relation.field2)` you can see two forms of spread access. `field1` is a *base spread access*, which applies directly to the context's base expression. Like a normal access expression, `base` specifies a block and the `field1` specifies the name of the field to read from it. `relation.field2` is a *chained spread access*, in which the left-hand side (`relation`) is applied to the context's base expression to select a block and then the right-hand side is used to select the field to read from it.

In [ ]:
cosim_blocks["attitude_pointing_mode"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        root!.(attitude, activePointingMode.name),
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Spread Tuples
A *spread tuple* `(spread1, spread2)` is a spread element that contains further spread elements, and applies them to the context's base expression to produce an inner tuple.

In [ ]:
cosim_blocks["spread2"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        root!.(
        position,
        (attitude, activePointingMode.name)),
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Inner Spreads
An *inner spread* `spread1.(spread2, spread3)` applies its left-hand side `spread1` to the context's base expression to produce an intermediate value, and then forms a tuple by applying each element of its right-hand side to the intermediate value.

In [ ]:
cosim_blocks["actual_attitude_commanded_attitude"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        root!.(
            attitude,
            activePointingMode.(
                name,
                commandedAttitude
            )
        ),
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Spread conversions
A *spread conversion* `spread as Kind.unit` applies its left-hand side `spread` to the context's base expression and then converts the resulting value to the unit or reference frame specified by `Kind.unit`.

In [ ]:
cosim_blocks["position_lla_attitude_eci_body"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        root!.(position as Position.lla, attitude as Quaternion.eci_body),
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

### Invariant Spreads
Some spread elements, such as constants and ephemerides, don't use the value of the base at all. These are mostly used in type matches.

In [ ]:
cosim_blocks["constant_spread"] = scenario.SpontaneousExternalState.create(
    consumed='''(
        root!.(0, "a string", null),
    )''',
    produced='''()''',
    engine='gnc',
    agents=[BASICSAT1_ID]
)

# Cosimulating

## Starting a simulation
Once you've set up the blocks on the simulation, you'll need to start the simulation in order to cosimulate. Once you start a simulation and receive a handle to it, you can run the spontaneous cosimulators to observe and modify the simulation state as it runs.

In [81]:
simulation_handle = scenario.simulation.start(wait=True, streamTelemetry=True)

## Consuming simulation data
For a cosimulator to consume data from the simulation, call the `consume` method on the simulation handle, passing the agent ID to consume data from and the ID of the external state block. This returns a Python object containing the data specified by the `consumed` query on the external state block.

In [ ]:
if simulation_handle.status()['status'] == 'RUNNING':
    # consume and unpack state from the simulation
    for name, block in cosim_blocks.items():
        state = simulation_handle.consume(agent_id=BASICSAT1_ID, external_state_id=block.id)
        print(f"{name}: {state}", flush=True)
else:
    print("Simulation not running!")